# Controllable generation via RL to let Elon Musk speak ill of DOGE
> How to control text generation through a sentiment classifier.



In [1]:
!pip install pfrl@git+https://github.com/voidful/pfrl.git
!pip install textrl==0.2.15

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/voidful/pfrl.git to /tmp/pip-install-svjh1d7h/pfrl_0ce69eae01084ad2bb298e823ab4dc31
  Running command git clone --filter=blob:none --quiet https://github.com/voidful/pfrl.git /tmp/pip-install-svjh1d7h/pfrl_0ce69eae01084ad2bb298e823ab4dc31
  Resolved https://github.com/voidful/pfrl.git to commit bd679022d91902a3c88066d2646ad69b30d5684a
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 3.2 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 32.7 MB/s eta 0:00:0000:0100:01
  Created wheel for pfrl: filename=pfrl-0.4.0-py3-none-any.whl size=155509 sha256=68b7a4d14352c5618e53ab0454422a7b0a6e0a49b5c67717678cb8d5079e505f
  Stored in directory: /tmp/pip-ephem-wheel-cache-gki01ezg/

In [2]:
from textrl import TextRLEnv,TextRLActor
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer, AutoModelWithLMHead
import logging
import sys
import pfrl
import torch
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')

/home/b0990106x/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Using a pre-trained model, it can generate elonmusk's style tweets.**

In [11]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("lca0503/speech-chatgpt-base-nar-v2-epoch4-wotrans")
model = AutoModelForSeq2SeqLM.from_pretrained("lca0503/speech-chatgpt-base-nar-v2-epoch4-wotrans")

# tokenizer = AutoTokenizer.from_pretrained("huggingtweets/elonmusk")  
# model = AutoModelWithLMHead.from_pretrained("huggingtweets/elonmusk")
model.eval()
model.cuda()

tokenizer_config.json: 100%|██████████| 349/349 [00:00<00:00, 1.24MB/s]
vocab.json: 100%|██████████| 798k/798k [00:00<00:00, 1.00MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 767kB/s]
tokenizer.json: 100%|██████████| 3.82M/3.82M [00:01<00:00, 3.20MB/s]
added_tokens.json: 100%|██████████| 211k/211k [00:00<00:00, 530kB/s]
special_tokens_map.json: 100%|██████████| 280/280 [00:00<00:00, 2.21MB/s]
config.json: 100%|██████████| 1.75k/1.75k [00:00<00:00, 5.80MB/s]
pytorch_model.bin: 100%|██████████| 586M/586M [00:31<00:00, 18.4MB/s] 


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(59481, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(59481, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

**a sentiment classifier for rl reward**

In [12]:
sentiment = pipeline('sentiment-analysis',model="cardiffnlp/twitter-roberta-base-sentiment",tokenizer="cardiffnlp/twitter-roberta-base-sentiment",device=0,return_all_scores=True)

/home/b0990106x/miniconda3/lib/python3.11/site-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [5]:
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.CRITICAL)

In [6]:
sentiment("dogecoin is bad")

[[{'label': 'LABEL_0', 'score': 0.933853268623352},
  {'label': 'LABEL_1', 'score': 0.0601189061999321},
  {'label': 'LABEL_2', 'score': 0.00602772505953908}]]

In [7]:
sentiment("dogecoin is bad")[0][0]['score']

0.933853268623352

set our text generation reward, inverse perplexity + sentiment classifier.
- inverse perplexity make sure the generated sentence probability will be high.
- sentiment classifier can make the generate more negative.

In [8]:
class MyRLEnv(TextRLEnv):
    def get_reward(self, input_item, predicted_list, finish): # predicted will be the list of predicted token
      reward = 0
      if finish or len(predicted_list) >= self.env_max_length:
        predicted_text = tokenizer.convert_tokens_to_string(predicted_list[0])
        # sentiment classifier
        reward = sentiment(input_item[0]+predicted_text)[0][0]['score'] * 10
      return reward

**fit one example**

In [9]:
observaton_list = [{'input':'i think dogecoin is'}]

In [11]:
env = MyRLEnv(model, tokenizer, observation_input=observaton_list,compare_sample=1)
actor = TextRLActor(env,model,tokenizer)
agent = actor.agent_ppo(update_interval=100, minibatch_size=3, epochs=10)

In [12]:
actor.predict(observaton_list[0])

[' a good idea<|endoftext|>']

In [ ]:
pfrl.experiments.train_agent_with_evaluation(
    agent,
    env,
    steps=300,
    eval_n_steps=None,
    eval_n_episodes=1,       
    train_max_episode_len=100,  
    eval_interval=10,
    outdir='elon_musk_dogecoin', 
)

loading the best result and predict.

In [16]:
agent.load("./elon_musk_dogecoin/best")

In [17]:
actor.predict(observaton_list[0])

[' a real thing<|endoftext|>']